In [ ]:
# For parsing and saving
import datetime as dt
import csv

from pprint import pprint

from util.helpers import startWebdriver
from util.log_videos import update_video_log

from util.custom_values import DATA_DIR, CHANNEL_ID, CHROMEDRIVER_PATH, USER_DATA_PATH, CHROME_PROFILE
from util.constants import METRICS, TimePeriod, TRAFFIC_SOURCES_IMP, \
    TRAFFIC_SOURCES, TRAFFIC_SOURCES_INV, Dimensions, ADV_URL, VIDEOS_URL, VIDEO_URL

In [ ]:
from util.api_scrape import scrape_channels_info

In [ ]:
driver = startWebdriver()
output = scrape_channels_info(driver, [CHANNEL_ID]*5)
output

TESTING API SCRAPING

In [ ]:
from util.api_scrape import switch_to_iframe, execute
from util.api_scrape_utils import clean_response, parse_video_item_str
import datetime as dt
from typing import List, Dict, Union

API_URL = "https://developers.google.com/youtube/v3/docs/{mode}/list?apix=true"

def scrape_videos_basics_by_api(driver, video_ids: List[str]) -> Dict[str, Dict[str,Union[str,dt.datetime]]]:
    """
    Return dictionary of video upload date and title by video id.
    """
    videos = {}

    try:
        driver.get(API_URL.format(mode="videos"))

        switch_to_iframe(driver)
        
        # Enter arguments
        driver.find_element(By.ID, "part[0]").send_keys("snippet") # Enter parts
        driver.find_element(By.CSS_SELECTOR, "label.mat-checkbox-layout").click() # Disable auth

        for vid_id in video_ids:
            # Enter playlist id
            element = driver.find_element(By.ID, f"id[0]")
            element.clear()
            element.send_keys(vid_id)

            # Get result in json string
            json_string = clean_response(execute(driver))

            # Parse the video json into our format
            video_dict = parse_video_item_str(json_string)
            if not video_dict:
                print(f"Couldn't find {vid_id}: might be private [{__file__}]")
                continue
            videos[vid_id] = video_dict
    except Exception as e:
        print("error whilst scraping video info from YouTube api: ", e, f"[{__file__}]")
    finally:
        # driver.quit()
        return videos

In [ ]:
driver = startWebdriver()
videos = scrape_videos_basics_by_api(driver, video_ids)

display(videos)

TESTING VIDEO LOG ENCODING ERROR

In [ ]:
import datetime
videos = {'h1MsqH5dxCU': {'datetime': datetime.datetime(2023, 8, 4, 0, 0),
  'title': 'ASMR Hand Sounds but with some ✨ Chaotic ✨ Energy (Obviously)',
  'precise': 0},
 '6SQdNjSUVuc': {'datetime': datetime.datetime(2023, 7, 26, 0, 0),
  'title': "Haircut ASMR but You're a Plant (Obviously)",
  'precise': 0},
 'qLIjpyVNORc': {'datetime': datetime.datetime(2022, 11, 25, 0, 0),
  'title': 'Cosy ASMR (Pillow Crinkles, Affirmations, Tapping, Whispers)',
  'precise': 0},
 'hzPs_owbyBc': {'datetime': datetime.datetime(2022, 10, 24, 0, 0),
  'title': '(ASMR) Spit Painting - Concerned Friend Roleplay (Obviously)',
  'precise': 0},
 'V1-mU98FIOU': {'datetime': datetime.datetime(2022, 10, 10, 0, 0),
  'title': '(ASMR) Personal Attention Obviously',
  'precise': 0}}
# Make list of videos
videos_list = [videos[video] for video in videos]
videos_list

In [ ]:
# Save videos to new csv file
with open("video_log.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, videos_list[0].keys())
    writer.writeheader()
    writer.writerows(videos_list)
    print(f"Written scraped video data to video_log.csv")